In [1]:
import os
import numpy as np
import glob
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.morphology import closing, disk

In [2]:
save_dir = "/projectnb/aicv/Shihua_Qin/BMLSeg2D/dataset/BMLClosing"
file_paths = glob.glob("/projectnb/aicv/Shihua_Qin/BMLSeg2D/dataset/*/*.bmp")
with tqdm(total=len(file_paths)) as pbar:
    for file_path in file_paths:
        file_dir = os.path.join(save_dir, file_path.split("/")[-2])
        os.makedirs(file_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "/".join(file_path.split("/")[-2:]))

        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if "labels" in file_dir:
            comb_path = os.path.join("/projectnb/aicv/Shihua_Qin/BMLSeg2D/dataset/BMLClosing/comb", os.path.split(file_path)[-1])
            footprint = disk(4)
            closed = closing(img, footprint)

            smoothed_closed = cv2.GaussianBlur(closed, (5, 5), 0)
            _, smoothed_closed = cv2.threshold(smoothed_closed, 127, 255, cv2.THRESH_BINARY)
            
            smoothed_closed[smoothed_closed>=125] = 255
            smoothed_closed[smoothed_closed<125] = 0
            
            comb = np.concatenate([img, smoothed_closed], axis=1)
            cv2.imwrite(save_path, smoothed_closed)
            cv2.imwrite(comb_path, comb)
        else:
            cv2.imwrite(save_path, img)
        
        pbar.update(1)

100%|██████████| 4248/4248 [02:05<00:00, 33.77it/s]


In [ ]:
bml_paths = sorted(glob.glob("/projectnb/aicv/Shihua_Qin/BMLSeg2D/dataset/BMLClosing/label*/*00.bmp"))
bone_paths = sorted(glob.glob("/projectnb/aicv/Shihua_Qin/BMLSeg2D/dataset/BMLClosing/label*/*01.bmp"))

with tqdm(total=len(bml_paths)) as pbar:
    for bml_path, bone_path in zip(bml_paths, bone_paths):
        bml = cv2.imread(bml_path, cv2.IMREAD_GRAYSCALE)
        bone = cv2.imread(bone_path, cv2.IMREAD_GRAYSCALE)
        bone_binary = (bone > 125).astype(np.uint8)
        modified_bml = np.multiply(bone_binary, bml)
        cv2.imwrite(bml_path, modified_bml)
        pbar.update(1)